In [1]:
import ROOT
import numpy as np

In [2]:
2/np.sqrt(3) *.1

np.float64(0.11547005383792518)

## Define Classes

In [3]:
class Diamond:
  def __init__(self, u:int, v:int, sidelength:float, angle:float=np.pi/3) -> None:
    self.x_ = [u,u+1,u+1,u,u] # The whole loop
    self.y_ = [v,v,v+1,v+1,v]
    self.ncorners = 4

    self.shearDiamond(angle)
    scale = lambda x:x*sidelength

    self.x_ = np.array(list(map(scale,self.x_)))
    self.y_ = np.array(list(map(scale,self.y_)))

  def shearDiamond(self,angle):
    cosA = np.cos(angle); sinA = np.sin(angle)
    shearMatrix = np.matrix([[1,-cosA],
                             [0,sinA]])
    for i in range(self.ncorners +1):
      xy = np.array(shearMatrix@np.array([self.x_[i],self.y_[i]]))[0]
      self.x_[i] = xy[0]
      self.y_[i] = xy[1]
    return

  def rotateDiamond(self, angle:float) -> None:
    cosA = np.cos(angle); sinA = np.sin(angle) 
    roatationMatrix = np.matrix([[cosA,-sinA],
                                 [sinA,cosA]])
    for i in range(self.ncorners +1):
      xy = np.array(roatationMatrix@np.array([self.x_[i],self.y_[i]]))[0]
      self.x_[i] = xy[0]
      self.y_[i] = xy[1]
    return
    
  def __repr__(self):
    out_lst = []
    for i in range(self.ncorners + 1):
      out_lst.append((self.x_[i],self.y_[i]))
    return str(out_lst)

  
class CoarseDiamond:
  def __init__(self,seq,side_length = 9.68 ): #9.8 cm the side length of a module
    self.side_length = side_length
    self.seq = [Diamond(ele[0],ele[1],self.side_length/4) for ele in seq]

  def rotateDiamond(self, angle:float) -> None:
    for diamond in self.seq:
      diamond.rotateDiamond(angle)
    return


In [4]:
D = Diamond(0,1,1)
D

[(np.float64(-0.5000000000000001), np.float64(0.8660254037844386)), (np.float64(0.4999999999999999), np.float64(0.8660254037844386)), (np.float64(-2.220446049250313e-16), np.float64(1.7320508075688772)), (np.float64(-1.0000000000000002), np.float64(1.7320508075688772)), (np.float64(-0.5000000000000001), np.float64(0.8660254037844386))]

## Creating Plots

### Partials

In [5]:
ML_F_0_seq = [(2,0), (3,0), (3,1), (2,1),
              (3,2),(3,3),(2,2),(2,3),
              (1,0),(1,1),(0,0),(0,1),
              (0,2),(1,2),(0,3),(1,3)]
CD_0 = CoarseDiamond(ML_F_0_seq)
CD_0.rotateDiamond(np.pi+np.pi/6)
gr = ROOT.TH2Poly()

for i,diamond in enumerate(CD_0.seq):
  bin = ROOT.TGraph(diamond.ncorners+1,diamond.x_,diamond.y_)
  gr.AddBin(bin)
  gr.SetBinContent(i+1,i)

if ROOT.gROOT.FindObject("canvas"):
    ROOT.gROOT.FindObject("canvas").Close()
canvas = ROOT.TCanvas('canvas',"iu iv coord example",600,600)

gr.Draw()
canvas.Draw()

In [6]:
ML_F_1_seq = [(2,0),(3,0),(3,1),(2,1),
              (3,2),(3,3),(2,2),(2,3),
              (1,0),(1,1),(0,0),(0,1),
              (0,2),(1,2),(0,3),(1,3)]

CD_1 = CoarseDiamond(ML_F_1_seq)
CD_1.rotateDiamond(np.pi/2)
gr = ROOT.TH2Poly()

for i,diamond in enumerate(CD_1.seq):
  bin = ROOT.TGraph(diamond.ncorners+1,diamond.x_,diamond.y_)
  gr.AddBin(bin)
  gr.SetBinContent(i+1,i+16)

if ROOT.gROOT.FindObject("canvas"):
    ROOT.gROOT.FindObject("canvas").Close()
canvas = ROOT.TCanvas('canvas',"iu iv coord example",600,600)

gr.Draw()
canvas.Draw()


In [7]:
ML_F_2_seq = [(2,0),(3,0),(3,1),(2,1),
              (3,2),(3,3),(2,2),(2,3),
              (1,0),(1,1),(0,0),(0,1),
              (0,2),(1,2),(0,3),(1,3)]

CD_2 = CoarseDiamond(ML_F_2_seq)
CD_2.rotateDiamond(-np.pi/6)
gr = ROOT.TH2Poly()

for i,diamond in enumerate(CD_2.seq):
  bin = ROOT.TGraph(diamond.ncorners+1,diamond.x_,diamond.y_)
  gr.AddBin(bin)
  gr.SetBinContent(i+1,i+16)

if ROOT.gROOT.FindObject("canvas"):
    ROOT.gROOT.FindObject("canvas").Close()
canvas = ROOT.TCanvas('canvas',"iu iv coord example",600,600)

gr.Draw()
canvas.Draw()

### Defining helping functions

In [8]:
out_dir = '/home/mijail/python_projects/CMS TPG/geometry_output/TCGeometries/'
deltairot = -5*np.pi/6

In [9]:
def create_wafer(TC_seq:list[Diamond], typecode:str, nc_cells:list[int] = []) -> None:
  file = ROOT.TFile(out_dir+'geometry_'+typecode+'_TC_wafer.root','RECREATE')
  gr = ROOT.TH2Poly()
  gr.SetName(typecode.replace('_','-'))

  file.mkdir("hex")
  file.cd("/hex")
  deltai = 0
  for i,diamond in enumerate(TC_seq):
    bins = ROOT.TGraph(diamond.ncorners+1,diamond.x_,diamond.y_)
    name = "TC_{}".format(i)
    if i in nc_cells:
      name += "_nc"
      deltai += 1
    bins.SetName(name)
    bins.Write()
    if i not in nc_cells: gr.AddBin(bins)
    gr.SetBinContent(i - deltai + 1, i)

  file.cd('/')
  gr.Write()
  file.Write()

## MH

#### MH-F

In [10]:
MH_F_seq = [(2,2),(3,3),(3,2),(2,3),
            (1,2),(0,3),(0,2),(1,3),
            (1,1),(0,0),(0,1),(1,0),
            (2,1),(3,0),(3,1),(2,0)]

CD_0 = CoarseDiamond(MH_F_seq)
CD_1 = CoarseDiamond(MH_F_seq)
CD_2 = CoarseDiamond(MH_F_seq)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(-np.pi/6 + deltairot)
CD_2.rotateDiamond(np.pi/2 + deltairot)

TC_seq = CD_0.seq + CD_1.seq + CD_2.seq
create_wafer(TC_seq,"ML_F")

In [11]:
MH_F_seq = [(2,2),(3,3),(3,2),(2,3),
            (1,2),(0,3),(0,2),(1,3),
            (1,1),(0,0),(0,1),(1,0),
            (2,1),(3,0),(3,1),(2,0)]

CD = CoarseDiamond(MH_F_seq)
CD.rotateDiamond(3*np.pi/6+deltairot)

file = ROOT.TFile(out_dir+'geometry_MH_F_TC_wafer.root','RECREATE')
gr = ROOT.TH2Poly()
gr.SetName("MH-F")

file.mkdir("hex")
file.cd("/hex")
for i in [0,1,2]:
  CD.rotateDiamond(2*np.pi/3)
  for j,diamond in enumerate(CD.seq):
    bins = ROOT.TGraph(diamond.ncorners+1,diamond.x_,diamond.y_)
    bins.SetName("TC_{}".format(16*i+j))
    bins.Write()
    gr.AddBin(bins)
    gr.SetBinContent(16*i + j + 1, 16*i + j)

if ROOT.gROOT.FindObject("canvas"):
    ROOT.gROOT.FindObject("canvas").Close()
canvas = ROOT.TCanvas('canvas',"iu iv coord example",600,600)
file.cd('/')
gr.Write()
gr.Draw()
canvas.Draw()
file.Write()
file.Close()

#### MH-L

In [12]:
typecode = "MH_L"
MH_L_0_seq = [(2,0),(3,1),(3,0),(3,2),
              (2,2),(3,3),(2,1),(1,0)]
MH_L_1_seq = [(0,2),(1,2),(0,1),(2,2),
              (0,3),(2,3),(1,3),(3,3)]

CD_0 = CoarseDiamond(MH_L_0_seq)
CD_1 = CoarseDiamond(MH_L_1_seq)

CD_0.rotateDiamond(-np.pi/6 + deltairot)
CD_1.rotateDiamond(-5*np.pi/6 + deltairot)

TC_seq = CD_0.seq + CD_1.seq  

create_wafer(TC_seq,typecode)

#### MH-R

In [13]:
typecode = "MH_R"
MH_R_0_seq = [(3,0),(3,2),(3,1),(3,3),
            (2,0),(2,1),(1,0),(2,2)]
MH_R_1_seq = [(0,1),(1,2),(0,2),(2,2),
              (0,3),(2,3),(1,3),(3,3)]
deltairot = -5*np.pi/6
CD_0 = CoarseDiamond(MH_R_0_seq)
CD_0.rotateDiamond(-7/6 *np.pi+deltairot)
CD_1 = CoarseDiamond(MH_R_1_seq)
CD_1.rotateDiamond(np.pi/6+deltairot)

TC_seq = CD_0.seq + CD_1.seq

gr = ROOT.TH2Poly()
gr.SetName(typecode.replace('_','-'))

file.mkdir("hex")
file.cd("/hex")

for i,diamond in enumerate(TC_seq):
  bins = ROOT.TGraph(diamond.ncorners+1,diamond.x_,diamond.y_)
  bins.SetName("TC_{}".format(i))
  gr.AddBin(bins)
  gr.SetBinContent(i + 1, i)

if ROOT.gROOT.FindObject("canvas"):
    ROOT.gROOT.FindObject("canvas").Close()
canvas = ROOT.TCanvas('canvas',"iu iv coord example",600,600)
file.cd('/')
gr.Draw()
canvas.Draw()
create_wafer(TC_seq,typecode)

#### MH-T

In [14]:
typecode = "MH_T"
MH_T_0_seq = [(3,2),(3,3),(2,2),(2,3),
              (1,1),(0,3),(1,2),(1,3),
              (2,0),(1,0),(3,1),(2,1),
              (3,0)]
MH_T_1_seq = [(0,2),(1,3),(0,3), #there are diamonds 8 10 11
              (0,1),(2,3),(1,2)]

CD_0 = CoarseDiamond(MH_T_0_seq)
CD_1 = CoarseDiamond(MH_T_1_seq)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(np.pi/2 + deltairot)

diamond8 = CD_1.seq.pop(0)
diamond10 = CD_1.seq.pop(0)
diamond11 = CD_1.seq.pop(0)

CD_0.seq.insert(8,diamond8)
CD_0.seq.insert(10,diamond10)
CD_0.seq.insert(11,diamond11)

TC_seq = CD_0.seq + CD_1.seq

create_wafer(TC_seq,typecode)

#### MH-B

In [15]:
typecode = "MH_B"
MH_B_0_seq = [(2,1),(3,0),(2,0),(3,1),
              (1,2),(2,2),(3,3),(3,2),
              (1,1),(1,0),(0,0)]
MH_B_1_seq = [(0,0),(2,2),(3,3),(3,2),
              (2,3),(1,2),(0,3),(0,2),
              (1,3),(2,1),(1,0),(1,1),
              (3,1),(2,0),(3,0),(0,1)]
MH_B_2_seq = [(0,0),(0,1),(0,2)]

CD_0 = CoarseDiamond(MH_B_0_seq)
CD_1 = CoarseDiamond(MH_B_1_seq)
CD_2 = CoarseDiamond(MH_B_2_seq)

CD_0.rotateDiamond(np.pi/2 + deltairot)
CD_1.rotateDiamond(-np.pi/6 + deltairot)
CD_2.rotateDiamond(-5*np.pi/6 + deltairot)

diamond8 = CD_1.seq.pop()

CD_NC = CoarseDiamond([(0,4),(4,0)])

TC_seq = CD_0.seq + CD_1.seq
TC_seq.insert(8,diamond8)
TC_seq.insert(12,CD_NC.seq[0])
TC_seq.insert(14,CD_NC.seq[1])
TC_seq.insert(15,CD_2.seq[0])
TC_seq.insert(27,CD_2.seq[1])
TC_seq.insert(31,CD_2.seq[2])

create_wafer(TC_seq,typecode,[12,14])

## ML

#### ML-F

In [16]:
ML_F_seq = [(2,0),(3,0),(3,1),(2,1),
            (3,2),(3,3),(2,2),(2,3),
            (1,0),(1,1),(0,0),(0,1),
            (0,2),(1,2),(0,3),(1,3)]

CD = CoarseDiamond(ML_F_seq)
CD.rotateDiamond(-np.pi/6 - 5*np.pi/6)

gr = ROOT.TH2Poly()
gr.SetName("ML-F")
for i in range(3):
  CD.rotateDiamond(-2*np.pi/3)
  for j,diamond in enumerate(CD.seq):
    bins = ROOT.TGraph(diamond.ncorners+1,diamond.x_,diamond.y_)
    bins.SetName("TC_{}".format(16*i+j))
    gr.AddBin(bins)
    gr.SetBinContent(16*i + j + 1, 16*i + j)

if ROOT.gROOT.FindObject("canvas"):
    ROOT.gROOT.FindObject("canvas").Close()
canvas = ROOT.TCanvas('canvas',"iu iv coord example",600,600)

gr.Draw()
canvas.Draw()

In [17]:
ML_F_seq = [(2,0),(3,0),(3,1),(2,1),
            (3,2),(3,3),(2,2),(2,3),
            (1,0),(1,1),(0,0),(0,1),
            (0,2),(1,2),(0,3),(1,3)]
          
CD_0 = CoarseDiamond(ML_F_seq)
CD_1 = CoarseDiamond(ML_F_seq)
CD_2 = CoarseDiamond(ML_F_seq)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(np.pi/2 + deltairot)
CD_2.rotateDiamond(-np.pi/6 + deltairot)

TC_seq = CD_0.seq + CD_1.seq + CD_2.seq
create_wafer(TC_seq,"ML_F")

In [18]:
typecode = "ML_F_old"
ML_F_seq_0 = [(0,3),(0,2),(1,3),(1,2),
            (0,1),(0,0),(1,1),(1,0),
            (2,3),(2,2),(3,3),(3,2),
            (2,1),(2,0),(3,1),(3,0)]

ML_F_seq_1 = [(0,0),(0,1),(1,1),(1,0),
            (0,2),(0,3),(1,3),(1,2),
            (2,2),(2,3),(3,3),(3,2),
            (2,0),(2,1),(3,1),(3,0)]

ML_F_seq_2 = [(3,1),(2,1),(3,0),(2,0),
            (1,1),(0,1),(1,0),(0,0),
            (3,3),(2,3),(3,2),(2,2),
            (1,3),(0,3),(1,2),(0,2)]

CD_0 = CoarseDiamond(ML_F_seq_0)
CD_1 = CoarseDiamond(ML_F_seq_1)
CD_2 = CoarseDiamond(ML_F_seq_2)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(np.pi/2 + deltairot)
CD_2.rotateDiamond(-np.pi/6 + deltairot)

TC_seq = CD_0.seq + CD_1.seq + CD_2.seq
create_wafer(TC_seq,typecode)

#### ML-5

In [19]:
typecode = "ML_5"
ML_5_0_seq = [(0,1),(0,0),(1,1),(1,0),
              (3,0),(2,0),(2,1),(3,1),
              (0,2),(0,3),(1,2),(1,3),
              (3,3),(3,2),(2,3),(2,2)]
ML_5_1_seq = [(0,0),(2,0),(1,0),(2,1),
              (0,1),(0,3),(0,2),(1,1),
              (3,0),(3,1),(3,2),(3,3),
              (1,2),(2,2),(1,3),(2,3)]
ML_5_2_seq = [(2,0),(3,0),(3,1),(2,1),
              (3,2),(3,3),(2,3),(2,2),
              (1,0),(0,0),(0,1),(1,1),
              (0,2),(1,2),(0,3),(1,3)]

CD_0 = CoarseDiamond(ML_5_0_seq)
CD_1 = CoarseDiamond(ML_5_1_seq)
CD_2 = CoarseDiamond(ML_5_2_seq)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(np.pi/2 + deltairot)
CD_2.rotateDiamond(-np.pi/6 + deltairot)

TC_seq = CD_0.seq + CD_1.seq + CD_2.seq

create_wafer(TC_seq,typecode,[i for i in range(24,32)])

#### ML-L

In [20]:
typecode = "ML_L"
ML_L_0_seq = [(0,3),(0,2),(0,1),(0,0),
              (1,2),(1,3),(2,3),(3,3),
              (2,2),(3,2),(2,1),(1,1)]
ML_L_1_seq = [(3,0),(2,0),(1,0),(0,0),
              (2,1),(3,1),(3,2),(3,3),
              (2,2),(2,3),(1,1),(1,2)]

CD_0 = CoarseDiamond(ML_L_0_seq)
CD_1 = CoarseDiamond(ML_L_1_seq)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(-np.pi/6 + deltairot)

diamond1 = CD_1.seq.pop(0)
diamond3 = CD_1.seq.pop(0)
diamond5 = CD_1.seq.pop(0)
diamond7 = CD_1.seq.pop(0)

TC_seq = CD_0.seq + CD_1.seq
TC_seq.insert(1,diamond1)
TC_seq.insert(3,diamond3)
TC_seq.insert(5,diamond5)
TC_seq.insert(7,diamond7)

create_wafer(TC_seq,typecode)

#### ML-R

In [21]:
typecode = "ML_R"
ML_R_0_seq = [(1,1),(2,1),(3,2),(3,3),
              (2,2),(2,3),(1,3),(1,2),
              (0,0),(0,1),(0,2),(0,3)]

ML_R_1_seq = [(0,0),(1,0),(2,0),(3,0),
              (1,2),(2,3),(2,2),(1,1),
              (2,1),(3,1),(3,2),(3,3)]

CD_0 = CoarseDiamond(ML_R_0_seq)
CD_1 = CoarseDiamond(ML_R_1_seq)

CD_0.rotateDiamond(np.pi/6 + deltairot)
CD_1.rotateDiamond(5*np.pi/6 + deltairot)

diamond9 = CD_1.seq.pop(0)
diamond11 = CD_1.seq.pop(0)
diamond12 = CD_1.seq.pop(0)
diamond14 = CD_1.seq.pop(0)

TC_seq = CD_0.seq + CD_1.seq

TC_seq.insert(9,diamond9)
TC_seq.insert(11,diamond11)
TC_seq.insert(12,diamond12)
TC_seq.insert(14,diamond14)

create_wafer(TC_seq,typecode)

#### ML-B

In [22]:
typecode = "ML_B"

ML_B_0_seq = [(0,1),(0,0),(1,1),(1,0),
              (2,1),(2,0),(3,0),(3,1),
              (1,2),(0,2),(0,3),(1,3),
              (2,3),(3,3),(2,2),(3,2)]

ML_B_1_seq = [(1,0),(2,0),(3,0),(3,1),
              (3,2),(3,3),(2,1),(2,2)]

CD_0 = CoarseDiamond(ML_B_0_seq)
CD_1 = CoarseDiamond(ML_B_1_seq)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(3*np.pi/6 + deltairot)

#### ML-T

In [23]:
typecode = "ML_T"

ML_T_0_seq = [(0,1),(0,0),(1,1),(1,0),
              (2,1),(2,0),(3,0),(3,1),
              (1,2),(0,2),(0,3),(1,3),
              (2,3),(3,3),(3,2),(2,2)]

ML_T_1_seq = [(0,1),(1,2),(2,3),
              (0,2),(1,3),
              (0,3)]

CD_0 = CoarseDiamond(ML_T_0_seq)
CD_1 = CoarseDiamond(ML_T_1_seq)

CD_0.rotateDiamond(-5*np.pi/6 + deltairot)
CD_1.rotateDiamond(3*np.pi/6 + deltairot)

TC_seq = CD_0.seq + CD_1.seq

create_wafer(TC_seq,typecode)